In [1]:
import requests
from bs4 import BeautifulSoup

headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36',
}
def getDSID(user_id = '*********', password = '*********'):    
    url = 'https://vpn.hit.edu.cn/dana-na/auth/url_default/login.cgi'
    post_data = {
        'tz_offset': '480',
        'username': user_id,
        'password': password,
        'realm': '学生',
        'btnSubmit': '登陆'
    }   
    print('第一次登陆')
    r = requests.post(url, data = post_data,  allow_redirects = False)
    cookies = r.headers.get('Set-Cookie')
    
    # 如果没有Set-Cookie字段，说明登陆失败，这是尝试把之前的登陆挤掉
    if (cookies == None):
        print('第一次登陆失败,重新登陆中...')
        r = requests.post(url, data = post_data,  allow_redirects = True)
        soup = BeautifulSoup(r.text)
        from_data_str = soup(id = 'DSIDFormDataStr')[0].get('value')
        post_data = {
            'btnContinue': '继续会话',
            'FormDataStr': from_data_str,
        }
        r = requests.post(url, data = post_data,  allow_redirects = False)
        cookies = r.headers.get('Set-Cookie')
        print('第二次登陆成功')
    else:
        print('第一次登陆成功')
    start_index = cookies.find('DSID')
    start_index = cookies.find('=', start_index)
    end_index = cookies.find(';', start_index)
    print(cookies[start_index+1:end_index])
    return cookies[start_index+1:end_index]
        
cookies_dsid = getDSID()

第一次登陆
第一次登陆成功
56a914ce806824d9c9043f52460dc5b5


In [3]:
# 将网页中的表格处理成为二维布尔数组
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import time
import functools
import json
import requests
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup


MAX_THREAD_SIZE = 8
MAX_CLASSROOM_NUMBER = 300
MAX_WEEK = 18
SQL_DIR = 'kjs_sql/'#程序初始化地时候检查文件夹是否存在，写入文件的时候不检查
MAX_THREAD_SIZE = 4
USE_VPN = True
TIMEOUT = 120
ALL_INFO_FILE = 'room-info.json'

url_vpn = 'https://vpn.hit.edu.cn/kjscx/,DanaInfo=jwts.hit.edu.cn+queryKjs_wdl'
url_0 = 'http://jwts.hit.edu.cn/kjscx/queryKjs_wdl'

def get_classroom_html(*args, **kwargs):
    week, camp_id, building_id = args[0], args[1], args[2]
    post_data = {
        'pageXnxq': '2019-20201',
        'pageZc1': str(week),
        'pageZc2': str(week),
        'pageXiaoqu': camp_id,
        'pageLhdm': building_id,
        'pageCddm': '',
        'pageSize': MAX_CLASSROOM_NUMBER,
    }
    r = None
    if 'use_vpn' in kwargs.keys() and not kwargs.get('use_vpn'):
#         print('no vpn')
        r = requests.post(url_0, data = post_data, timeout = TIMEOUT)
    else:
        cookies = {
            'DSID': cookies_dsid,
        }
        r = requests.post(url_vpn, data = post_data, headers = headers, cookies = cookies)
    soup = BeautifulSoup(r.text)
    soup = soup('table', 'dataTable')
#     print(soup)
    if (len(soup) == 0):
        print('获取网页失败，可能是会话过期或者网站已更新')
        return None
    else:
        return week, soup[0]


def table2array(soup):
    # 将表格中的一行处理为布尔数组，被table2array调用
    def tr2bool_list(tr_soup):
        tr_soup = tr_soup('td')[1:]
        ite = map(lambda td_cell: 'kjs_icon' in td_cell('div')[0].get('class'), tr_soup)
        return list(ite)
    
    soup = soup('tr')[2:]
    ite = map(lambda tr_soup: tr2bool_list(tr_soup), soup)
    return list(ite)


# 并发地获取一栋教学楼的1-18周的空教室信息，并且存储为文件


def write2file(building_id, info_list):
    building_id = ('%03d' % (int(building_id)))
    classroom_size = len(info_list)
    file_path = SQL_DIR + 'room-info-' + building_id + '.json'
    with open(file_path, 'a') as f:
        for i in range(classroom_size):
            str_row = '{\"_id\":\"' + building_id + '-' + ('%03d' % (i+1)) + '\", \"info\": ['
            for j in range(MAX_WEEK * 7 * 6):
                if j != 0:
                    str_row += ','
                if info_list[i][j]:
                    str_row += 'true'
                else:
                    str_row += 'false'
            str_row += ']\n'
            f.write(str_row)

def get_building_info(camp_id:str, building_id:str, building_name:str):
    kjs_progress = tqdm_notebook(range(MAX_WEEK),desc=building_name)
    info_list = [None] * MAX_WEEK
    def process_html(future):
        week, soup = future.result()
#         print(week)
        info_list[week-1]= table2array(soup)
        kjs_progress.update(1)
        
    with concurrent.futures.ThreadPoolExecutor(MAX_THREAD_SIZE) as pool:
        for i in range(MAX_WEEK):
            temp = pool.submit(get_classroom_html, i+1, camp_id, building_id, use_vpn = USE_VPN)
            temp.add_done_callback(process_html)
        pool.shutdown(wait=True)
        
    classroom_size = len(info_list[0])
    
    def merge_list(l1, l2):
        for i in range(classroom_size):
            l1[i] = l1[i] + l2[i]
        return l1
    
    info_list = functools.reduce(merge_list, info_list)
    return info_list
# temp_info_list = get_building_info('1', '016', '正心楼')
def main():
    with open(SQL_DIR + 'f1.json', 'r', encoding='utf-8') as f:
        all_buildings = json.load(f)
        for camp in all_buildings:
            camp_id = str(camp['index'])
            for building in camp['buildings']:
                building_id, building_name = building['id'], building['name']
                info_list = get_building_info(camp_id, building_id, building_name)
                write2file(building_id, info_list)
        with open(SQL_DIR + ALL_INFO_FILE, 'w') as fa:
            for camp in all_buildings:
                camp_id = str(camp['index'])
                for building in camp['buildings']:
                    building_id = ('%03d' % (int(building['id'])))
                    file_path = SQL_DIR + 'room-info-' + building_id + '.json'
                    with open(file_path, 'r') as f:
                        fa.write(f.read())
# main()

In [6]:
import functools
def parse_room_name(tab_soup):
    tab_soup = tab_soup[2:]
    return list(map(lambda x: x('td')[0].text, tab_soup))

def parse_floor(name_list, max_floor = 10, offset = 0):
    floor_list = list([] for x in range(max_floor + 1))
    index = 0
    for room_name in name_list:
        length = len(room_name)
        floor, temp_floor, index = 0, 0, index + 1
        start = offset
        while room_name[start] < '1' or room_name[start] > '9':
            start += 1
            if (start + 1 >= length):
                break
        end = start + 1
        while end < length:
            if room_name[end] >= '0' and room_name[end] <= '9':
                end += 1
            else:
                break
        if room_name[start] >= '0' and room_name[start] <= '9':
            temp_floor = int(room_name[start])
            floor = temp_floor
            if start + 2 <= end:
                temp_floor *= 10
                temp_floor += int(room_name[start+1])
                if temp_floor <= max_floor and end - start >= 4:
                    floor = temp_floor
        floor_list[floor].append((index, room_name))
    return floor_list

def write2file_floor(building_id, floor_list):
    building_id = ('%03d' % (int(building_id)))
    file_path = SQL_DIR + building_id + '.json'
    with open(file_path, 'w', encoding='utf-8') as f:
        for i in range(len(floor_list)):
            if len(floor_list[i]) == 0:
                continue
            str_row, room_id, room_name = {}, [], []
            str_row['_id'] = building_id + '-f' + str(i)
            for ele in floor_list[i]:
                room_id.append(building_id + '-%03d' % (ele[0]))
                room_name.append(ele[1])
            str_row['room-id'] = room_id
            str_row['room-name'] = room_name
            str_row = json.dumps(str_row, ensure_ascii=False)
#             print(str_row)
            f.write(str_row+ '\n', )

def get_floor():
    with open(SQL_DIR + 'f2.json', 'r', encoding='utf-8') as f:
        for xq in json.loads(f.read()):
            camp_id = xq['index']
            for building in xq['buildings']:
                info_list = get_classroom_html(1, camp_id, building['id'], use_vpn = USE_VPN)
                info_list = parse_room_name(info_list[1]('tr'))
                info_list = parse_floor(info_list)
                write2file_floor(building['id'], info_list)
                print(building['name'])

get_floor()
# fl = parse_floor(rl)
# write2file_floor('999', fl)

正心楼
主楼
诚意楼
格物楼
理学楼
致知楼
管理学院楼
学士楼
新技术楼
机械楼
材料楼
电机楼
管理楼
主楼
东配楼
西配楼
车库楼
青年公寓
理化楼
